In [22]:
"""
    Author : Tyronne Wickramarathne
    Date   : 03-11-2021
    Description : This Python code would insert Google Mobility Community data and Covid 19 data from the available countries into a MongoDB database.

    E mail :
"""
import pandas as pd
import numpy as np
import warnings
from pymongo import MongoClient
from pprint import pprint
from tabulate import tabulate
warnings.filterwarnings('ignore')

In [23]:
cd = pd.read_csv("~/nfs/data-science/data/owid-covid-data.csv",low_memory=False)
cd.head().transpose()

,0,1,2,3,4
iso_code,AFG,AFG,AFG,AFG,AFG
continent,Asia,Asia,Asia,Asia,Asia
location,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan
date,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28
total_cases,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...
human_development_index,0.511,0.511,0.511,0.511,0.511
excess_mortality_cumulative_absolute,NaN,NaN,NaN,NaN,NaN
excess_mortality_cumulative,NaN,NaN,NaN,NaN,NaN
excess_mortality,NaN,NaN,NaN,NaN,NaN


In [24]:
md = pd.read_csv("~/nfs/data-science/data/google-mobility.csv",low_memory=False)
md.head().transpose()

,0,1,2,3,4
country_region_code,AE,AE,AE,AE,AE
country_region,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates,United Arab Emirates
sub_region_1,NaN,NaN,NaN,NaN,NaN
sub_region_2,NaN,NaN,NaN,NaN,NaN
metro_area,NaN,NaN,NaN,NaN,NaN
iso_3166_2_code,NaN,NaN,NaN,NaN,NaN
census_fips_code,NaN,NaN,NaN,NaN,NaN
place_id,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM,ChIJvRKrsd9IXj4RpwoIwFYv0zM
date,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19
retail_and_recreation_percent_change_from_baseline,0.0,1.0,-1.0,-2.0,-2.0


In [25]:
client = MongoClient()

In [26]:
md_stripped = md.drop(columns=['metro_area','iso_3166_2_code','census_fips_code','place_id'])
md_stripped.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [27]:
md_stripped = md_stripped.rename(columns={'country_region_code': 'Country_Code', 'country_region': 'Country','sub_region_1':'State','sub_region_2':'Suburb','date':'Date','retail_and_recreation_percent_change_from_baseline':'Retail_Recreation','grocery_and_pharmacy_percent_change_from_baseline':'Grocery_Pharmacy','parks_percent_change_from_baseline':'Parks','transit_stations_percent_change_from_baseline':'Transit','workplaces_percent_change_from_baseline':'Workplace','residential_percent_change_from_baseline':'Residential'})

#Creating a database, 'pandemic' to host pandemic data
db = client["pandemic"]

#Inserting Google Community Mobility Data to the table 'mobility'
mobility = db["mobility"]
result = mobility.insert_many(md_stripped.to_dict('records'))

#Inserting Covid Cases data to the table 'covid'
covid = db["covid"]
result = covid.insert_many(cd.to_dict('records'))

client.close()